In [24]:
#Dependencies
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Create a reference the CSV file desired
csv_path = "FARS2016NationalCSV/accident.csv"

# Read the CSV into a Pandas DataFrame
accidents = pd.read_csv(csv_path)


,STATE,ST_CASE,VE_TOTAL,VE_FORMS,PVH_INVL,PEDS,PERNOTMVIT,PERMVIT,PERSONS,COUNTY,...,NOT_MIN,ARR_HOUR,ARR_MIN,HOSP_HR,HOSP_MN,CF1,CF2,CF3,FATALS,DRUNK_DR
0,1,10001,1,1,0,0,0,1,1,73,...,31,9,36,88,88,0,0,0,1,1
1,1,10002,1,1,0,0,0,2,2,73,...,99,20,5,99,99,0,0,0,1,1
2,1,10003,2,1,1,0,1,1,2,73,...,99,2,31,88,88,0,0,0,1,0
3,1,10004,1,1,0,0,0,1,1,73,...,99,21,57,99,99,0,0,0,1,1
4,1,10005,1,1,0,0,0,1,1,73,...,7,20,10,88,88,0,0,0,1,0


In [3]:
accidents_df = accidents.drop(accidents.columns[[1,2,4,5,6,8,17,18,19,20,24,25,26,27,30,31,32,33,39,40,41,42,43,44,45,46]], axis=1)

,STATE,VE_FORMS,PERMVIT,COUNTY,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,...,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR
0,1,1,1,73,1716,1,1,2016,6,9,...,5,1,1,0,1,0,0,0,1,1
1,1,1,2,73,1716,10,1,2016,1,19,...,4,3,1,0,1,0,0,0,1,1
2,1,1,1,73,0,1,1,2016,6,2,...,4,3,10,0,10,0,0,0,1,0
3,1,1,1,73,0,17,1,2016,1,21,...,3,3,1,0,1,0,0,0,1,1
4,1,1,1,73,3125,26,1,2016,3,20,...,5,3,2,0,2,0,0,0,1,0


In [ ]:
csv_path1 = "FARS2016NationalCSV/states.csv"
states = pd.read_csv(csv_path1)

In [4]:
accidents_df['STATE'] = accidents_df['STATE'].map(states.set_index('Codes')['State'])

,STATE,VE_FORMS,PERMVIT,COUNTY,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,...,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR
0,Alabama,1,1,73,1716,1,1,2016,6,9,...,5,1,1,0,1,0,0,0,1,1
1,Alabama,1,2,73,1716,10,1,2016,1,19,...,4,3,1,0,1,0,0,0,1,1
2,Alabama,1,1,73,0,1,1,2016,6,2,...,4,3,10,0,10,0,0,0,1,0
3,Alabama,1,1,73,0,17,1,2016,1,21,...,3,3,1,0,1,0,0,0,1,1
4,Alabama,1,1,73,3125,26,1,2016,3,20,...,5,3,2,0,2,0,0,0,1,0


In [5]:
xls_path = "GLC.xlsx"
xl = pd.ExcelFile(xls_path)
state_codes = xl.parse("Sheet 1")
state_codes.head()

,Territory,State Name,State Abbreviation,State Code,City Name/County Name,City Code,County Code
0,U,ALABAMA,AL,1.0,ABBEVILLE,10.0,67.0
1,U,ALABAMA,AL,1.0,ALBERTVILLE,50.0,95.0
2,U,ALABAMA,AL,1.0,ALEXANDER CITY,60.0,123.0
3,U,ALABAMA,AL,1.0,ANDALUSIA,90.0,39.0
4,U,ALABAMA,AL,1.0,ANDERSON,96.0,77.0


In [6]:
state_codes.fillna(0, inplace=True)

In [7]:
state_codes["City Code"]= state_codes["City Code"].astype(int)
state_codes["County Code"]= state_codes["County Code"].astype(int)
state_codes["State Name"] = state_codes["State Name"].astype(str) 

In [8]:
state_codes = state_codes.apply(lambda x: x.astype(str).str.title())

In [9]:
state_codes["state_county_city"] = state_codes["State Name"].map(str) + "_" + state_codes["County Code"].map(str) + "_" + state_codes["City Code"].map(str)

In [10]:
accidents_df["state_county_city"] = accidents_df["STATE"].map(str) + "_" + accidents_df["COUNTY"].map(str) + "_" + accidents_df["CITY"].map(str)

In [11]:
state_codes.loc[state_codes.duplicated('state_county_city', keep=False)]

,Territory,State Name,State Abbreviation,State Code,City Name/County Name,City Code,County Code,state_county_city
1048,U,Alaska,Ak,2.0,Denali,0,68,Alaska_68_0
1049,U,Alaska,Ak,2.0,Denali Borough,0,68,Alaska_68_0
46087,0,0,0,0.0,0,0,0,0_0_0
46088,For Official Use Only.,0,0,0.0,0,0,0,0_0_0


In [12]:
state_codes = state_codes.drop(state_codes.index[[1049,46088]])


In [13]:
accidents_df['CITY'] = accidents_df['state_county_city'].map(state_codes.set_index('state_county_city')['City Name/County Name'])

In [14]:
accidents_df.head()

,STATE,VE_FORMS,PERMVIT,COUNTY,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,...,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR,state_county_city
0,Alabama,1,1,73,Hoover,1,1,2016,6,9,...,1,1,0,1,0,0,0,1,1,Alabama_73_1716
1,Alabama,1,2,73,Hoover,10,1,2016,1,19,...,3,1,0,1,0,0,0,1,1,Alabama_73_1716
2,Alabama,1,1,73,Jefferson,1,1,2016,6,2,...,3,10,0,10,0,0,0,1,0,Alabama_73_0
3,Alabama,1,1,73,Jefferson,17,1,2016,1,21,...,3,1,0,1,0,0,0,1,1,Alabama_73_0
4,Alabama,1,1,73,Vestavia Hills,26,1,2016,3,20,...,3,2,0,2,0,0,0,1,0,Alabama_73_3125


In [15]:
csv_path2 = "FARS2016NationalCSV/weather.csv"
csv_path3 = "FARS2016NationalCSV/CF.csv"
csv_path4 = "FARS2016NationalCSV/light.csv"
csv_path5 = "FARS2016NationalCSV/road.csv"
weather = pd.read_csv(csv_path2)
cf = pd.read_csv(csv_path3)
light_cond = pd.read_csv(csv_path4)
road = pd.read_csv(csv_path5)

In [16]:
accidents_df['WEATHER'] = accidents_df['WEATHER'].map(weather.set_index('Code')['Weather'])
accidents_df['WEATHER1'] = accidents_df['WEATHER1'].map(weather.set_index('Code')['Weather'])
accidents_df['WEATHER2'] = accidents_df['WEATHER2'].map(weather.set_index('Code')['Weather'])

In [17]:
accidents_df['CF1'] = accidents_df['CF1'].map(cf.set_index('Code')['Attribute'])
accidents_df['CF2'] = accidents_df['CF2'].map(cf.set_index('Code')['Attribute'])
accidents_df['CF3'] = accidents_df['CF3'].map(cf.set_index('Code')['Attribute'])

In [18]:
accidents_df['LGT_COND'] = accidents_df['LGT_COND'].map(light_cond.set_index('Code')['Attribute'])

In [19]:
accidents_df['REL_ROAD'] = accidents_df['REL_ROAD'].map(road.set_index('Code')['Attribute'])

In [21]:
accidents_df = accidents_df.drop(accidents_df.columns[[3,26]], axis=1)

In [2]:
accidents_df

NameError: name 'accidents_df' is not defined

In [1]:
x = accidents_df["CF1"].value_counts()
x

NameError: name 'accidents_df' is not defined